In [1]:
import pandas as pd
import os
import unittest
import string
import re
pd.set_option('display.max_rows', 10)

In [2]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day14_input.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s = file1.read().splitlines()
pd.Series(s)

0      mask = 100110001110110011001X101110X1XX10X1
1                             mem[62998] = 9708340
2                           mem[33440] = 124867031
3                                mem[6869] = 81126
4      mask = 11110000X01010010XX10X000XX00X001010
                          ...                     
575                             mem[65299] = 41354
576                               mem[42445] = 501
577    mask = 01111X110010000X01001100X0000000001X
578                              mem[28748] = 8402
579                               mem[9833] = 7040
Length: 580, dtype: object

In [3]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day14_example1.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s_ex1 = file1.read().splitlines()
pd.Series(s_ex1)

0    mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
1                                    mem[8] = 11
2                                   mem[7] = 101
3                                     mem[8] = 0
dtype: object

In [4]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day14_example2.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file2 = open(trainFile, 'r') 
s_ex2 = file2.read().splitlines()
pd.Series(s_ex2)

0    mask = 000000000000000000000000000000X1001X
1                                  mem[42] = 100
2    mask = 00000000000000000000000000000000X0XX
3                                    mem[26] = 1
dtype: object

In [5]:
def combine(value, mask):    
    result = (value | mask[1]) & ~mask[0]
    #print(value, mask[1], mask[0], value | mask[1], (value | mask[1]) & ~mask[0])
    return result

In [6]:
def parseAssignment(line):
    parts = line.split("=")
    left = parts[0][3:]
    return (int(left.strip("mem[] ")), int(parts[1]))

In [7]:
def parseMask(line):
    text = line[7:].strip(" ")
    andValue = 0
    orValue = 0
    floatingValue = 0
    for i in range(0,36):
        bit = 1 << i
        if text[36-i-1] == "0":
            andValue = andValue | bit
        elif text[36-i-1] == "1":
            orValue = orValue | bit            
        elif text[36-i-1] == "X":
            floatingValue = floatingValue | bit
    return (andValue, orValue, floatingValue)

In [8]:
def process(input):
    memory = {}
    mask = 0
    for line in input:
        if line.startswith("mask"):
            mask = parseMask(line)
            #print(mask)
        elif line.startswith("mem"):
            command = parseAssignment(line)
            #print(command)
            memory[command[0]] = combine(command[1],mask)

    #print(memory)
    return sum(memory.values())

In [9]:
process(s_ex1)

165

In [10]:
process(s)

14839536808842

In [11]:
def expand(values, bit):
    result = []
    for v in values:
        result.append(v)
        result.append(v | bit)
    return result

def createFloatingCombinations(floatingValue):
    result = [0]
    for i in range(0,36):
        bit = 1 << i
        if bit & floatingValue != 0:
            result = expand(result[:],bit)
    return result

In [12]:
def createAddressList(baseAddress, mask):
    #print(baseAddress, mask)
    result = []
    andValue = mask[0]
    orValue = mask[1]
    floatingValue = mask[2]
    combinations = createFloatingCombinations(floatingValue)
    for c in combinations:
        address = ((baseAddress | orValue)  & ~floatingValue) | c
        result.append(address)
    return result

In [13]:
def processPartTwo(input):
    memory = {}
    mask = 0
    for line in input:
        if line.startswith("mask"):
            mask = parseMask(line)
        elif line.startswith("mem"):
            command = parseAssignment(line)
            
            addresses = createAddressList(command[0], mask)
            for a in addresses:
                memory[a] = command[1]

            #print(memory)
    return sum(memory.values())

In [14]:
createFloatingCombinations(5)

[0, 4, 1, 5]

In [15]:
processPartTwo(s_ex2)

208

In [16]:
processPartTwo(s)

4215284199669